In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import os

import spacy
from IPython.display import display

from src.pipelinelib.querying import Parser, Queryable

folder = os.path.join(os.getcwd(), "data", "transcripts")
files = [os.path.join(root, f) for root, _, files in os.walk(folder)
         for f in files if f.endswith(".docx")]


nlp = spacy.load("de_core_news_sm", disable=["ner", "parser"])
parser = Parser(
    nlp=nlp, metadata_path="./data/transcripts/Kopie von Transkriptionspaare_Daten.xls")

parser.read_from_files(files)
display(parser.frame)


=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 81_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 87_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 182_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 138_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 47_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 105_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 29_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 27_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 58_T1_IM_F

,document_id,paragraph_id,sentence_id,couple_id,speaker,gender,is_depressed_group,depressed_person,text
0,0,0,0,81,A,W,False,None,Ich dachte wir reden darüber wie wir die erste...
1,0,1,1,81,B,M,False,None,Erste Wohnung?
2,0,2,2,81,A,W,False,None,"Ja, ich dachte das ist ein nettes Thema, oder?"
3,0,3,3,81,B,M,False,None,Ja.
4,0,4,4,81,A,W,False,None,"Ich fand das süß, wie du erst angefangen hast ..."
...,...,...,...,...,...,...,...,...,...
126,9,94,126,60,B,M,True,W,Hab ich ein Schild auf dem Rücken?
127,9,94,127,60,B,M,True,W,Ladet eure Probleme bei mir ab.
128,9,95,128,60,A,W,True,W,Das sind keine Probleme.
129,9,96,129,60,B,M,True,W,Kümmerst du dich drum.


In [50]:
from src.pipelinelib.querying import Parser, Queryable
from src.pipelinelib.text_body import TextBody
from src.pipelinelib.pipeline import Pipeline
queryable = Queryable.from_parser(parser)

In [56]:
#from src.sigmund import adapter
from src.sigmund.extensions import TALKTURN

from src.sigmund.features import agreement_score as fagree
from src.sigmund.features import flesch_reading_ease as fflesch
from src.sigmund.features import talk_turn as ftalkturn
from src.sigmund.features import liwc_one_hot as fliwconehot
from src.sigmund.features import pos as fpos
from src.sigmund.preprocessing import words as pwords

liwc_dict_path = r"/home/rise/Schreibtisch/Sigmund_git/sigmund/data/German_LIWC2001_Dictionary.dic"

pipeline = Pipeline(queryable = queryable, empty_pipeline=True) \
    .add_component(pwords.Tokenizer())\
    .add_component(fpos.PartOfSpeech())\
    .add_component(fliwconehot.LiwcOneHot())\
    .add_component(ftalkturn.TalkTurnExtractor())\
    .add_component(fagree.AgreementScoreExtractor())\
    .add_component(fflesch.FleschExtractor())

    #.add_component(fliwconehot.LiwcOneHot())
    #.add_component(adapter.Adapter(old = TOKENS_sese, new = TALKTURN))\


SyntaxError: unexpected character after line continuation character (<ipython-input-56-78c338c81196>, line 16)

In [44]:
test = pipeline.execute()

=== Starting pipeline with ['Tokenizer', 'PartOfSpeech'] ===
Executing Tokenizer
=== Queryable is executing on Document level, query = '' ===
=== Queryable is executing on Paragraph level, query = '' ===
=== Queryable is executing on Sentence level, query = '' ===
Executing PartOfSpeech
=== Queryable is executing on Document level, query = '' ===
=== Finished pipeline execution ===


In [46]:
display(test)

{<src.pipelinelib.extension.Extension at 0x7f388d220880>:      couple_id speaker gender  is_depressed_group  document_id  paragraph_id  \
 0           81       A      W               False            0             0   
 1           81       B      M               False            0             1   
 2           81       A      W               False            0             2   
 3           81       B      M               False            0             3   
 4           81       A      W               False            0             4   
 ..         ...     ...    ...                 ...          ...           ...   
 126         60       B      M                True            9            94   
 127         60       B      M                True            9            94   
 128         60       A      W                True            9            95   
 129         60       B      M                True            9            96   
 130         60       B      M                True  

In [12]:
document_df = queryable.execute(level=TextBody.DOCUMENT)
display(document_df)

=== Queryable is executing on Document level, query = '' ===


,document_id,couple_id,is_depressed_group,depressed_person,text
0,0,81,False,None,Ich dachte wir reden darüber wie wir die erste...
1,1,87,False,None,"Wie lang?. 10 Minuten. Okay, dann fang mal an...."
2,2,182,False,None,Hm?. Du solltest doch nicht nachgucken.. Sag n...
3,3,138,True,W,"Am Ammersee.. Dann machen wir es so.. Da, wo m..."
4,4,47,True,W,Erzähl mir was.. Also wir müssen die Fragen je...
5,5,105,True,W,Jetzt sollen wir uns unterhalten?. Genau.. Al...
6,6,29,False,None,"Gut, fangen wir mal an.. Ja, fang du mal an.. ..."
7,7,27,True,W,"Positive Gefühle.. Ja, positive Gefühle können..."
8,8,58,False,None,Soll ich anfangen?. Ich hab mir was überlegt.....
9,9,60,True,W,"Wenn´s positiv geht, dann wird es nicht laut, ..."


In [ ]:
paragraph_df = queryable.execute(level=TextBody.PARAGRAPH)
display(paragraph_df)

In [ ]:
sentence_df = queryable.execute(level=TextBody.SENTENCE)
display(sentence_df)

queryable.execute(level=TextBody.SENTENCE)

In [ ]:
pgs = queryable.by_couple_id(couple_id=27) \
    .is_depressed(d=False) \
    .execute(level=TextBody.DOCUMENT)
display(pgs)